##  Chameleon Facility Port

This notebook demonstrates how to stitch experiment spanning [Chameleon](https://www.chameleoncloud.org/) and FABRIC.

### Set Chameleon Environment

FABRIC should already be configured in your Jupyter environment. If this is your first time using FABRIC, may need to follow the [Configure Environment](../../fablib_api/configure_environment/configure_environment.ipynb) notebook to complete the configuration.

Set the following environment vars to the values found in your custom Chameleon-openrc.sh file. You can obtain your Chameleon-openrc.sh from Chameleon using these [directions](https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html#the-openstack-rc-script).  The expected password is not your regular Chameleon password. Instead, you need to create a Chameleon CLI password by following these [directions](https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html#cli-authentication)

In [ ]:
import os

os.environ["OS_USERNAME"]='pruth'
os.environ["USER"]='pruth'

os.environ["OS_PASSWORD"]='Ren72jUGvWMq'
os.environ["OS_PROJECT_ID"]="ae76673270164b048b59d3bd30676721" #UC
#os.environ["OS_PROJECT_ID"]="a400724e818d40cbba1a5c6b5e714462" #TACC



os.environ["OS_AUTH_URL"]='https://chi.uc.chameleoncloud.org:5000/v3'
os.environ["OS_IDENTITY_API_VERSION"]='3'
os.environ["OS_INTERFACE"]='public'
os.environ["OS_PROTOCOL"]="openid"
os.environ["OS_AUTH_TYPE"]="v3oidcpassword"
os.environ["OS_IDENTITY_PROVIDER"]="chameleon"
os.environ["OS_DISCOVERY_ENDPOINT"]="https://auth.chameleoncloud.org/auth/realms/chameleon/.well-known/openid-configuration"
os.environ["OS_CLIENT_ID"]="keystone-uc-prod"
os.environ["OS_ACCESS_TOKEN_TYPE"]="access_token"
os.environ["OS_CLIENT_SECRET"]="none"
os.environ["OS_REGION_NAME"]="CHI@UC"


In [ ]:
import os

os.environ["OS_USERNAME"]='pruth'
os.environ["USER"]='pruth'

os.environ["OS_PASSWORD"]='Ren72jUGvWMq'
#os.environ["OS_PROJECT_ID"]="ae76673270164b048b59d3bd30676721" #UC
os.environ["OS_PROJECT_ID"]="a400724e818d40cbba1a5c6b5e714462" #TACC



os.environ["OS_AUTH_URL"]='https://chi.tacc.chameleoncloud.org:5000/v3'
os.environ["OS_IDENTITY_API_VERSION"]='3'
os.environ["OS_INTERFACE"]='public'
os.environ["OS_PROTOCOL"]="openid"
os.environ["OS_AUTH_TYPE"]="v3oidcpassword"
os.environ["OS_IDENTITY_PROVIDER"]="chameleon"
os.environ["OS_DISCOVERY_ENDPOINT"]="https://auth.chameleoncloud.org/auth/realms/chameleon/.well-known/openid-configuration"
os.environ["OS_CLIENT_ID"]="keystone-tacc-prod"
os.environ["OS_ACCESS_TOKEN_TYPE"]="access_token"
os.environ["OS_CLIENT_SECRET"]="none"
os.environ["OS_REGION_NAME"]="CHI@TACC"


### Import Libraries

In [ ]:
#General imports
import os
import json
import traceback
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
from datetime import datetime, timedelta
from dateutil import tz
import time

# Chameleon Library
import chi
import chi.lease 
from chi.server import *
from chi.lease import *
from chi.network import *

# FABRIC Library
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

### Chameleon Experiment Variable

In [ ]:
# Chameleon Config
chameleon_prefix =  "fabric_stitch_frr"
chameleon_server_name = chameleon_prefix+'Server'
chameleon_network_name = chameleon_prefix+'Net'
chameleon_subnet_name = chameleon_prefix+'Subnet'
chameleon_router_name = chameleon_prefix+'Router'
chameleon_lease_name = chameleon_prefix+'Lease'

chameleon_image_name='CC-Ubuntu20.04'
chameleon_node_type="compute_cascadelake_r"
chameleon_physical_network='physnet1'
chameleon_stitch_provider='fabric'
chameleon_server_count=1
chameleon_key_name='my_chameleon_key'

### FABRIC Experiment Variables

In [ ]:
# Create a FABlib manager
fablib = fablib_manager()
fablib.show_config()

# FABRIC Config
fabric_slice_name='chameleon_stitch_frr'
fabric_node_name='node1'

fabric_node_image='default_rocky_8'
fabric_site='MICH' #fablib.get_random_site()
print(f'fabric_site: {fabric_site}')

### Network Variables

The stitched network spans both Chameleon and FABRIC. The network needs a common subnet but distinct IP address allocation pools.

In [ ]:
#Network Config
subnet = IPv4Network("192.168.3.0/24")

fabric_allocation_pool_start=IPv4Address('192.168.3.2')
fabric_allocation_pool_end=IPv4Address('192.168.3.50')
fabric_available_ips=[]
for ip_int in range(int(fabric_allocation_pool_start),int(fabric_allocation_pool_end)+1):
    fabric_available_ips.append(IPv4Address(ip_int))
    
chameleon_allocation_pool_start='192.168.3.100'
chameleon_allocation_pool_end='192.168.3.150'
chameleon_gateway_ip='192.168.3.250'

## Create the Chameleon Network

#### Create a Lease at Chicago

In [ ]:
BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
#chi.lease.add_node_reservation(reservation_list, count=chameleon_server_count, node_type=chameleon_node_type)
#chi.lease.add_fip_reservation(reservation_list, count=2)

reservation_list.append(
        {
            "resource_type": "network",
            "network_name": chameleon_network_name,
            "network_properties": "",
            "resource_properties": json.dumps(
                ["==", "$stitch_provider", chameleon_stitch_provider]
            ),
        }
)

# Create the lease
chameleon_lease = chi.lease.create_lease(chameleon_lease_name,
                                  reservations=reservation_list,
                                  start_date=start_date,
                                  end_date=end_date)
    
#Print the lease info
#print(json.dumps(chameleon_lease, indent=2))
#chameleon_compute_reservation_id = [reservation for reservation in chameleon_lease['reservations'] if reservation['resource_type'] == 'physical:host'][0]['id']
chameleon_network_reservation_id = [reservation for reservation in chameleon_lease['reservations'] if reservation['resource_type'] == 'network'][0]['id']
#chameleon_floatingip_reservation_id = [reservation for reservation in chameleon_lease['reservations'] if reservation['resource_type'] == 'virtual:floatingip'][0]['id']

#print(f"chameleon_compute_reservation_id: {chameleon_compute_reservation_id}")
print(f"chameleon_network_reservation_id: {chameleon_network_reservation_id}")
#print(f"chameleon_floatingip_reservation_id: {chameleon_floatingip_reservation_id}")

#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The [get_network_by_name](../modules-python/network/get_network_by_name.ipynb) call will fail if a network with that name does not yet exits. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

In [ ]:
network_vlan = None
while network_vlan == None:
    try:
        #Get the network
        chameleon_network = chi.network.get_network(chameleon_network_name)

        #Get the network ID
        chameleon_network_id = chameleon_network['id']
        print(f'Chameleon Network ID: {chameleon_network_id}')

        #Get the VLAN tag (needed for FABRIC stitching)
        network_vlan = chameleon_network['provider:segmentation_id']
        print(f'network_vlan: {network_vlan}')
    except:
        print(f'Chameleon Network is not ready. Trying again!')
        time.sleep(10)           

#### Add a subnet to the reserved network

In [ ]:
chameleon_subnet = chi.network.create_subnet(chameleon_subnet_name, chameleon_network_id, 
                                             cidr=str(subnet),
                                             allocation_pool_start=chameleon_allocation_pool_start,
                                             allocation_pool_end=chameleon_allocation_pool_end,
                                             gateway_ip=chameleon_gateway_ip)

print(json.dumps(chameleon_subnet, indent=2))

#### Add a Router

In [ ]:
chameleon_router = chi.network.create_router(chameleon_router_name, gw_network_name='public')

print(json.dumps(chameleon_router, indent=2))

#### Attach the Router and Subnet

In [ ]:
chi.network.add_subnet_to_router_by_name(chameleon_router_name, chameleon_subnet_name)

## Start Chameleon Servers

In [ ]:
#import chi.server
for i in range(chameleon_server_count):
    server_name=f"{chameleon_server_name}_{i}"
    # Create the server
    server = chi.server.create_server(server_name, 
                                  reservation_id=chameleon_compute_reservation_id, 
                                  network_name=chameleon_network_name, 
                                  image_name=chameleon_image_name,
                                  key_name=chameleon_key_name
                                 )
# Wait until the server is active
#chi.server.wait_for_active(server.id)

## Get the Chameleon Server Fixed IPs

In [ ]:
#get fixed ips
fixed_ips={}
for i in range(chameleon_server_count):
    server_name=f"{chameleon_server_name}_{i}"
    server_id = get_server_id(server_name)
    fixed_ip = get_server(server_id).interface_list()[0].to_dict()["fixed_ips"][0]["ip_address"]
    fixed_ips[server_name]=fixed_ip

for server_name,fixed_ip in fixed_ips.items():
    print(f'{server_name}: {fixed_ip}')

## Test the Link by Pinging All Chameleon Nodes

In [ ]:
for server_name,fixed_ip in fixed_ips.items():
    print(f'{server_name}: {fixed_ip}')
    
    stdout, stderr = fabric_node.execute(f'ping -c 5 {fixed_ip}')
    print (stdout)
    print (stderr)

# Clean Up 

## Delete Chameleon Resources

Delete the servers

In [ ]:
for i in range(chameleon_server_count):
    server_name=f"{chameleon_server_name}_{i}"
    chi.server.delete_server(get_server_id(server_name))

#### De-configure Network

In [ ]:
router_id = chameleon_router['id']
subnet_id = chameleon_subnet['id']

try:
    result = chi.network.remove_subnet_from_router(router_id, subnet_id)
except Exception as e:
    print(f"detach_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_router(router_id)
except Exception as e:
    print(f"delete_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_subnet(subnet_id)
except Exception as e:
    print(f"delete_subnet_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_network(network_id)
except Exception as e:
    print(f"delete_network_by_name error: {str(e)}")
    pass

## Release Lease

In [ ]:
chi.lease.delete_lease(chameleon_lease['id'])

## Delete FABRIC Slice

In [ ]:
try:
    fabric_slice.delete()
except Exception as e:
    print(f"Exception: {e}")